# RKNN Conversion Guide

----------------------------

### Before you start

If you are not using Google Colab, it is recommended to create a separate [Python virtual environment](https://docs.python.org/3/library/venv.html) before you run this project. This ensures that packages installed for the conversion process do not conflict with other packages you may already have set up.

## ⚠️ Linux Only
This notebook can only by run on **Linux** because the `rknn-toolkit2` Python package only supports Linux builds.
If you don’t have access to a Linux system, consider using a cloud service like [Google Colab](https://colab.research.google.com).


### Setup

This notebook requires the use of custom Python code. Please run the installation script below to import these external scripts if you do not have them already.

**You may need to run the `Create ONNX/RKNN` cell multiple times when you restart your session. If you see a `create_onnx`
or `create_rknn` not found error, rerun the cell below and then retry.**

**Do not modify the cells in this setup section unless you know what you’re doing or have a specific reason to.**

#### Create ONNX/RKNN

Please run the cell below to be able to use the `create_onnx` and `create_rknn` functions.

In [ ]:
import os.path
import subprocess
import sys
import random

# This will work for all models that don't use anchors (e.g. all YOLO models except YOLOv5/v7)
# This includes YOLOv5u
yolo_non_anchor_repo = "https://github.com/airockchip/ultralytics_yolo11"

# For original YOLOv5 models
yolov5_repo = "https://github.com/airockchip/yolov5"

valid_yolo_versions = ["yolov5", "yolov8", "yolov11"]

ultralytics_folder_name_yolov5 = "airockchip_yolo_pkg_yolov5"
ultralytics_default_folder_name = "airockchip_yolo_pkg"

bad_model_msg = """
This is usually due to passing in the wrong model version.
Please make sure you have the right model version and try again.
"""


def print_bad_model_msg(cause):
    print(f"{cause}{bad_model_msg}")

def run_and_exit_with_error(cmd, error_msg, enable_error_output=True):
    try:
        if enable_error_output:
            subprocess.run(
                cmd,
                stderr=subprocess.STDOUT,
                stdout=subprocess.PIPE,
                universal_newlines=True,
            ).check_returncode()
        else:
            subprocess.run(cmd).check_returncode()
    except subprocess.CalledProcessError as e:
        print(error_msg)

        if enable_error_output:
            print(e.stdout)

        sys.exit(1)


def check_git_installed():
    run_and_exit_with_error(
        ["git", "--version"],
        """Git is not installed or not found in your PATH.
Please install Git from https://git-scm.com/downloads and try again.""",
    )


def check_or_clone_rockchip_repo(repo_url, repo_name=ultralytics_default_folder_name):
    if os.path.exists(repo_name):
        print(
            f'Existing Rockchip repo "{repo_name}" detected, skipping installation...'
        )
    else:
        print(f'Cloning Rockchip repo to "{repo_name}"')
        run_and_exit_with_error(
            ["git", "clone", repo_url, repo_name],
            "Failed to clone Rockchip repo, please see error output",
        )


def run_pip_install_or_else_exit(args):
    print("Running pip install...")
    run_and_exit_with_error(
        ["pip", "install"] + args,
        "Pip install rockchip repo failed, please see error output",
    )


def run_onnx_conversion_yolov5(model_path):
    check_or_clone_rockchip_repo(yolov5_repo, ultralytics_folder_name_yolov5)
    run_pip_install_or_else_exit(
        [
            "-r",
            os.path.join(ultralytics_folder_name_yolov5, "requirements.txt"),
            "torch<2.6.0",
            "onnx==1.18.0",
            "onnxscript",
        ]
    )

    model_abspath = os.path.abspath(model_path)

    try:
        subprocess.run(
            [
                "python",
                f"{ultralytics_folder_name_yolov5}/export.py",
                "--weights",
                model_abspath,
                "--rknpu",
                "--include",
                "onnx",
            ],
            stderr=subprocess.STDOUT,
            stdout=subprocess.PIPE,
            universal_newlines=True,
        ).check_returncode()
    except subprocess.CalledProcessError as e:
        print("Failed to run YOLOv5 export, please see error output")

        if "ModuleNotFoundError" in e.stdout and "ultralytics" in e.stdout:
            print_bad_model_msg(
                "It seems the YOLOv5 repo could not find an ultralytics installation."
            )
        elif "AttributeError" in e.stdout and "_register_detect_seperate" in e.stdout:
            print_bad_model_msg("It seems that you received a model attribute error.")
        else:
            print("Unknown Error when converting:")
            print(e.stdout)

        sys.exit(1)


def run_onnx_conversion_no_anchor(model_path):
    check_or_clone_rockchip_repo(yolo_non_anchor_repo)
    run_pip_install_or_else_exit(
        ["-e", ultralytics_default_folder_name, "onnx==1.18.0", "onnxscript"]
    )

    sys.path.insert(0, os.path.abspath(ultralytics_default_folder_name))
    model_abs_path = os.path.abspath(model_path)

    from ultralytics import YOLO

    try:
        model = YOLO(model_abs_path)
        model.export(format="rknn")
    except TypeError as e:
        if "originally trained" in str(e):
            print_bad_model_msg(
                "Ultralytics has detected that this model is a YOLOv5 model."
            )
        else:
            raise e

        sys.exit(1)


def create_onnx(model_path: str, version: str):
    check_git_installed()

    if not version in valid_yolo_versions:
        print(f"YOLO version \"{version}\" is not a valid version! Valid versions are: {', '.join(valid_yolo_versions)}")

    try:
        if version.lower() == "yolov5":
            run_onnx_conversion_yolov5(model_path)
        else:
            run_onnx_conversion_no_anchor(model_path)

        print(
            "Model export finished. Please use the generated ONNX file to convert to RKNN."
        )
    except SystemExit:
        raise RuntimeError("Model export failed. Please see output above.")

# RKNN Conversion code

image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp")
DEFAULT_PLATFORM = "rk3588"


def list_img_dir(img_dir):
    return [
        os.path.abspath(os.path.join(img_dir, f))
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_extensions)
    ]


def sample_imgs(num, img_list):
    if len(img_list) < num:
        return img_list
    else:
        return random.sample(img_list, num)


def get_image_list_from_dataset(num_imgs, yaml_dir):
    print(f"Dataset detected with {yaml_dir} file")
    img_raw_paths = []

    with open(yaml_dir, "r") as yaml_file:
        for line in yaml_file:
            line = line.strip()
            if (
                    line.startswith("train:")
                    or line.startswith("val:")
                    or line.startswith("test:")
            ):
                img_raw_paths.append(line.split(":", 1)[1].strip())

    no_yaml_dir = yaml_dir.replace(
        "data.yaml", "dummy_dir"
    )  # data.yaml sets dirs one level up
    img_set_paths = []

    for img_raw_path in img_raw_paths:
        p = (
            img_raw_path
            if os.path.isabs(img_raw_path)
            else os.path.realpath(os.path.join(no_yaml_dir, img_raw_path))
        )

        if os.path.exists(p):
            img_set_paths.append(p)

    if len(img_set_paths) < 1:
        return None

    all_imgs = [list_img_dir(path) for path in img_set_paths]

    for imgs in all_imgs:
        print(len(imgs))

    total_imgs = sum(len(group) for group in all_imgs)

    sampled_imgs = [
        sample_imgs(round((len(group) / total_imgs) * num_imgs), group)
        for group in all_imgs
    ]

    return [img for group in sampled_imgs for img in group]


def get_image_list_from_img_dir(num_imgs, img_dir):
    return sample_imgs(num_imgs, list_img_dir(img_dir))


def get_image_list(num_imgs, image_dir):
    yaml_path = os.path.join(image_dir, "data.yaml")

    if os.path.exists(yaml_path):
        return get_image_list_from_dataset(num_imgs, yaml_path)
    else:
        return get_image_list_from_img_dir(num_imgs, image_dir)


def run_rknn_conversion(
        img_list_txt, disable_quant, model_path, rknn_output, verbose_logging
):
    from rknn.api import RKNN

    rknn = RKNN(
        verbose=verbose_logging,
        verbose_file=("rknn_convert.log" if verbose_logging else None),
    )

    rknn.config(
        mean_values=[[0, 0, 0]],
        std_values=[[255, 255, 255]],
        target_platform=DEFAULT_PLATFORM,
    )

    print("Attempted RKNN load")
    ret = rknn.load_onnx(model=model_path)
    if ret != 0:
        print("Loading model failed!")
        exit(ret)

    print("Attempting RKNN build")
    ret = rknn.build(do_quantization=(not disable_quant), dataset=img_list_txt)
    if ret != 0:
        print("Building model failed!")
        exit(ret)

    print("Build succeeded! Starting export...")
    ret = rknn.export_rknn(rknn_output)
    if ret != 0:
        print("Exporting model failed!")
        exit(ret)
    print("Finished export!")

    # Release
    rknn.release()

    print(f'Your model is in "{rknn_output}" and ready to use!')


def create_rknn(
        model_path: str,
        rknn_output: str = "out.rknn",
        num_imgs: int = 300,
        img_dir: str = None,
        img_dataset_txt: str = "imgs.txt",
        disable_quantize: bool = False,
        verbose: bool = False,
):
    if not rknn_output.endswith(".rknn"):
        print("RKNN output path must end in .rknn!")
        return

    if not disable_quantize:
        if img_dir is None or len(img_dir) < 1:
            print(f"Must specify list of images to use with --img_dir")
            return

        img_dir_abs = os.path.abspath(img_dir)

        img_list = get_image_list(num_imgs, img_dir_abs)
        img_list_len = 0 if img_list is None else len(img_list)

        if img_list_len == 0:
            print(f"No images found in {img_dir_abs}")
            return
        elif img_list_len < num_imgs:
            print(
                f"Not enough images in your dataset/directory, you have {img_list_len} images, but need {num_imgs}"
            )
            return

        if not img_dataset_txt.endswith(".txt"):
            print(f"Image dataset text file path must end in .txt")
            return

        with open(img_dataset_txt, "w") as set_file:
            set_file.writelines(f"{img}\n" for img in img_list)

    try:
        run_rknn_conversion(
            img_dataset_txt,
            disable_quantize,
            model_path,
            rknn_output,
            verbose,
        )
    except SystemExit:
        print("RKNN Conversion failed, see output above")


#### *Numpy Fix* - Important for Google Colab Users

Google Colab comes with an incompatible version of Numpy installed. To fix this, please run the following cells below and **restart your session** when prompted.

In [ ]:
%pip uninstall numpy -y
%pip install "numpy>=1.23.0,<2.0.0"

### Step 1: Convert to ONNX

To convert to ONNX, simply run the `create_onnx` function, providing the path to your model weights and specifying the model version, as shown below.

In [ ]:
create_onnx(model_path="weights.pt", version="yolov8") # Valid versions are yolov5, yolov8, and yolov11

### Step 2: Download RKNN API

You can either use `pip` below to automatically detect and install the correct RKNN API Python library for you, or install it manually.

#### Automatic installation

Please run `pip` below. If it does not work, refer to the instructions for manual installation. You may need to restart your session after running the command below.


In [ ]:
%pip install rknn-toolkit2

#### Manual Installation (If Automatic Installation Fails)
Visit the [RKNN Toolkit 2](https://github.com/airockchip/rknn-toolkit2) Github repository, then click on rknn-toolkit2, followed by packages.
If you are running an x86_64 CPU (e.g., most Intel and AMD processors), select that option; otherwise, choose arm64 for ARM-based computers (e.g., M-series Macs or Snapdragon processors). If you're unsure which CPU you're using, check your system settings for processor architecture information.

Once you've selected the correct CPU architecture, you'll see multiple packages. The file names will look something like:
`rknn_toolkit2-2.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl`.
The numbers after cp correspond to your Python version. For example, if you're using Python 3.10, look for a package with cp310 in the name. For Python 3.8, look for cp38; for Python 3.7, cp37, and so on.

Once you've found the correct package, click the "Raw" button to download the .whl file. Then, run the following command in your terminal, replacing rknn_toolkit2.whl with the actual path to the file you downloaded:

In [ ]:
%pip install rknn_toolkit2.whl

### Step 3: Convert to RKNN

Please review the notes about quantization before running the RKNN conversion.

### Quantization

When performing quantization, it is critical to provide representative images of the objects or scenes you are trying to detect. These images are used to calibrate the model’s internal activations and greatly influence the final performance.

It is recommended to use 300–500 representative images that reflect the real-world input your model will encounter. As the old saying goes, it’s quality over quantity — having a diverse, relevant set matters more than simply having many images.

Quantization will cause some loss in model accuracy. However, if your calibration images are chosen wisely, this accuracy drop should be minimal and acceptable. If the sampled images are too uniform or unrelated, your quantized model's performance may worsen significantly.

The script will automatically sample representative images randomly from the provided dataset. While this usually works well, please verify that the dataset contains diverse and relevant examples of your target objects. As a reminder, the images used to quantize the model are stored in the text file specified by `--img_dataset_txt`.


### Optional: Download a dataset from Roboflow for quantization

If you do not already have a dataset or set of images containing the objects you want to detect, follow the steps below to download one from Roboflow Universe.

#### **Step 1: Search for a Dataset**

Go to [Roboflow Universe](https://universe.roboflow.com) and use the search bar to locate a dataset relevant to what you want to detect.
**Note:** The dataset must include the classes or object types you intend to detect.

#### **Step 2: Access the Dataset Tab**

After selecting a suitable project, navigate to the **Dataset** tab. Click the **"Download Dataset"** button. A prompt will appear with several options, including:

- Train a model with this dataset
- Train from a portion of this dataset
- Download dataset

Select **Download dataset**.

#### **Step 3: Choose Format and View Download Code**

- Under **Image and Annotation Format**, choose the version of YOLO you are using:
  - For **YOLOv5**, choose `YOLOv5 PyTorch`
  - For **YOLOv8**, choose `YOLOv8`
  - For **YOLOv11**, choose `YOLOv11`
- If multiple annotation formats are listed for your model, always select the one ending in **"PyTorch"**.

Then, under **Download Options**, click **"Show Download Code"** and continue.

In the resulting screen, you will see three tabs:
- **Jupyter**
- **Terminal**
- **Raw URL**

Select the **Terminal** tab and copy the provided command.

#### **Step 4: Paste and Run**

Paste the copied command into the notebook cell below and run it. This will download and extract the dataset into your environment, making it ready for use in the quantization process.

Make sure to prefix the command with "`!`" so it executes properly in this Jupyter Notebook environment.

In [ ]:
!curl -L "https://universe.roboflow.com/ds/FaF3HbDmF7?key=iMoJR25O9H" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

### RKNN Conversion Script
To get started, run the `create_rknn` script below, replacing the arguments with your own values. Refer to the table below for detailed information on each argument’s purpose and usage. The `model_path` argument should point to your exported ONNX model from Step 1, and `img_dir` must reference a valid directory containing either a dataset or a set of images to be used for quantization.

#### Overview of the `create_rknn` function

This script converts a YOLO ONNX model to RKNN format using a set of calibration images. It's designed to work with either:

- A flat directory of images (e.g. `train/images`), **or**
- A dataset directory containing a `data.yaml` file that defines `train`, `val`, and/or `test` folders.

##### Arguments

| Argument | Type | Description                                                                                                     |
|----------|------|-----------------------------------------------------------------------------------------------------------------|
| `img_dir` | `str` (required) | Path to your image directory. This can either be a folder of images **or** a dataset folder with a `data.yaml`. |
| `model_path` | `str` (required) | Path to your YOLO ONNX model, created in Step 1.                                                                |
| `num_imgs` | `int` (default: `300`) | Number of images to use for quantization calibration.                                                           |
| `disable_quantize` | `bool` (default: `False`) | Set to `True` to skip quantization entirely. Not recommended for performance, and should not be used for deployment on PhotonVision, which requires quantization.                                  |
| `rknn_output` | `str` (default: `out.rknn`) | File path where the final RKNN model should be saved.                                                           |
| `img_dataset_txt` | `str` (default: `imgs.txt`) | File path to store the list of images used during quantization.                                                 |
| `verbose` | `bool` (default: `False`) | Enable detailed logging from the RKNN API during conversion.                                                    |


##### *Notes*

1. This script is designed for use with [PhotonVision](https://photonvision.org), and by default sets the target platform for RKNN conversion to `RK3588`, a chipset commonly found in many variants of the Orange Pi 5 series (e.g., Orange Pi 5, 5 Pro, 5 Plus, 5 Max, etc.). You may modify the `DEFAULT_PLATFORM` value in the setup cell to match your specific hardware or deployment requirements if necessary.

2. If you followed the Roboflow dataset download instructions from the previous section, the dataset will have been extracted to your **current working directory**. In that case, you can simply set `img_dir` to "`.`" to reference the current directory.

In [ ]:
create_rknn(img_dir="./datasets", model_path="weights.onnx")